In [1]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os
#from dotenv import load_dotenv #

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [3]:
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
#import base64
import vertexai

from vertexai.generative_models import GenerativeModel, Part, FinishReason,GenerationConfig
import vertexai.preview.generative_models as generative_models

In [5]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [82]:
generation_config =GenerationConfig(
    temperature=0,
    top_p=0.5,
    max_output_tokens=8192,
)

In [7]:
model = GenerativeModel("gemini-1.5-flash-001")

DF2 es el csv que trajo santi tiene info hasta el 17/09

In [98]:
df2=pd.read_csv('info_solovenex_tweets.csv')

In [99]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      1134 non-null   object
 1   text      1112 non-null   object
 2   date      1134 non-null   object
 3   user      1134 non-null   object
 4   likes     1134 non-null   int64 
 5   quotes    1134 non-null   int64 
 6   retweets  1134 non-null   int64 
 7   comments  1134 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 71.0+ KB


In [100]:
df2.head()

,link,text,date,user,likes,quotes,retweets,comments
0,https://twitter.com/SOLOVENEX_/status/18367880...,‘𝗦𝗔𝗩𝗔’: ¡𝗘𝗟 𝗠𝗔́𝗦 𝗩𝗔𝗟𝗜𝗢𝗦𝗢! 🇻🇪 @QSavarino10 fue...,"Sep 19, 2024 · 3:22 PM UTC",@SOLOVENEX_,7,0,0,0
1,https://twitter.com/LAFC/status/18366166134682...,62' | GOAAALLL DAVID MARTÍNEZ #LAFCvATX 1-1,"Sep 19, 2024 · 4:01 AM UTC",@LAFC,184,1,15,11
2,https://twitter.com/SOLOVENEX_/status/18367318...,𝗗𝗔𝗩𝗜𝗗 𝗠𝗔𝗥𝗧𝗜́𝗡𝗘𝗭 anotó este miércoles su segund...,"Sep 19, 2024 · 11:39 AM UTC",@SOLOVENEX_,8,0,0,0
3,https://twitter.com/SOLOVENEX_/status/18364583...,🇻🇪 𝗝𝗨𝗔𝗡 𝗔𝗥𝗔𝗡𝗚𝗢 𝗝𝗥 (𝟮𝟬𝟬𝟲) escribió su nombre en...,"Sep 18, 2024 · 5:32 PM UTC",@SOLOVENEX_,16,0,4,0
4,https://twitter.com/SOLOVENEX_/status/18361468...,¡Ya son 𝟭𝟬𝟬 batallas con el Parma! 🇻🇪 El bari...,"Sep 17, 2024 · 8:54 PM UTC",@SOLOVENEX_,1026,5,139,5


El scrapeo mas actual, pocos tweets de diferencia

In [139]:
df=pd.read_csv('info_solovenex.csv') ##uso el actual 23/09

In [140]:
df.head(30)

,link,text,date,user,likes,quotes,retweets,comments
0,https://twitter.com/SOLOVENEX_/status/18376187...,Somos venezolanos. ¡Somos hermanos! 𝘼𝙍𝘼𝙈𝘽𝙐𝙍𝙐...,"Sep 21, 2024 · 10:23 PM UTC",@SOLOVENEX_,775,1,76,3
1,https://twitter.com/SOLOVENEX_/status/18378831...,No dejemos ir este momento (😂) ¡Comenta si t...,"Sep 22, 2024 · 3:54 PM UTC",@SOLOVENEX_,2,0,1,0
2,https://twitter.com/SOLOVENEX_/status/18376607...,𝗝𝗛𝗢𝗡𝗗𝗘𝗥 𝗖𝗔́𝗗𝗜𝗭 continúa siendo factor diferenc...,"Sep 22, 2024 · 1:10 AM UTC",@SOLOVENEX_,252,0,26,0
3,https://twitter.com/clubleonfc/status/18376489...,¡GOOOOOOOOOOOOOOOL DE LA FIEEEEERA! ¡GRÍTALO ...,"Sep 22, 2024 · 12:23 AM UTC",@clubleonfc,370,4,20,18
4,https://twitter.com/SOLOVENEX_/status/18375086...,“𝗔 𝗺𝗶 𝗺𝗲 𝗽𝗮𝗿𝗲𝗰𝗲 𝘂𝗻𝗮 𝗶𝗿𝗿𝗲𝘀𝗽𝗼𝗻𝘀𝗮𝗯𝗶𝗹𝗶𝗱𝗮𝗱 𝘀𝘂𝗯𝗶𝗿 𝗮 ...,"Sep 21, 2024 · 3:06 PM UTC",@SOLOVENEX_,280,18,60,30
5,https://twitter.com/SOLOVENEX_/status/18376187...,| 📸 @RealSociedad |,"Sep 21, 2024 · 10:23 PM UTC",@SOLOVENEX_,1,0,0,0
6,https://twitter.com/SOLOVENEX_/status/18376141...,🇻🇪 𝗝𝗢𝗡 𝗠𝗜𝗞𝗘𝗟 𝗔𝗥𝗔𝗠𝗕𝗨𝗥𝗨 sigue completando buenas...,"Sep 21, 2024 · 10:05 PM UTC",@SOLOVENEX_,23,0,3,0
7,https://twitter.com/SOLOVENEX_/status/18373174...,¡𝗥𝗢𝗡𝗗𝗢́𝗡 𝗘𝗡 𝗦𝗨 𝗦𝗔𝗟𝗦𝗔! 🇻🇪 Así marcó su 11º gol...,"Sep 21, 2024 · 2:26 AM UTC",@SOLOVENEX_,85,1,13,1
8,https://twitter.com/SOLOVENEX_/status/18372361...,👴🏻,"Sep 20, 2024 · 9:03 PM UTC",@SOLOVENEX_,1,0,0,1
9,https://twitter.com/carlostaracher/status/1837...,A quienes tenemos años trabajando para que cos...,"Sep 20, 2024 · 8:31 PM UTC",@carlostaracher,601,0,92,8


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      1142 non-null   object
 1   text      1120 non-null   object
 2   date      1142 non-null   object
 3   user      1142 non-null   object
 4   likes     1142 non-null   int64 
 5   quotes    1142 non-null   int64 
 6   retweets  1142 non-null   int64 
 7   comments  1142 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 71.5+ KB


In [11]:
df['text'].head()

0    Somos venezolanos. ¡Somos hermanos!   𝘼𝙍𝘼𝙈𝘽𝙐𝙍𝙐...
1    No dejemos ir este momento (😂)   ¡Comenta si t...
2    𝗝𝗛𝗢𝗡𝗗𝗘𝗥 𝗖𝗔́𝗗𝗜𝗭 continúa siendo factor diferenc...
3    ¡GOOOOOOOOOOOOOOOL DE LA FIEEEEERA!  ¡GRÍTALO ...
4    “𝗔 𝗺𝗶 𝗺𝗲 𝗽𝗮𝗿𝗲𝗰𝗲 𝘂𝗻𝗮 𝗶𝗿𝗿𝗲𝘀𝗽𝗼𝗻𝘀𝗮𝗯𝗶𝗹𝗶𝗱𝗮𝗱 𝘀𝘂𝗯𝗶𝗿 𝗮 ...
Name: text, dtype: object

In [ ]:
#si quiero pasar a date

# Supongamos que tu DataFrame se llama df y la columna se llama 'date'
#df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y · %I:%M %p %Z', errors='coerce')


In [137]:
#df = pd.read_csv('C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\scrapingcongemini\\venex\\info_solovenex.csv', header=None)
df = df.astype(str)  # Convert all values to strings
cadena_unica1 = ' '.join(df.values.flatten())
print(cadena_unica1)

https://twitter.com/SOLOVENEX_/status/1837618723538014486#m Somos venezolanos. ¡Somos hermanos!   𝘼𝙍𝘼𝙈𝘽𝙐𝙍𝙐 🔗 𝙈𝘼𝘾𝙃𝙄́𝙎 = 🫂❤️🇻🇪  #VenEx Sep 21, 2024 · 10:23 PM UTC @SOLOVENEX_ 775 1 76 3 https://twitter.com/SOLOVENEX_/status/1837883159553208575#m No dejemos ir este momento (😂)   ¡Comenta si tienes un pana así! ⬇️  | 🎥 Vía: @Lavinotintopod | Sep 22, 2024 · 3:54 PM UTC @SOLOVENEX_ 2 0 1 0 https://twitter.com/SOLOVENEX_/status/1837660758093844989#m 𝗝𝗛𝗢𝗡𝗗𝗘𝗥 𝗖𝗔́𝗗𝗜𝗭 continúa siendo factor diferencial con el Club León en la Liga MX y este sábado definió la victoria 1-0 sobre Atlético San Luis.   El delantero caraqueño disputó 66’ minutos y anotó su 𝗤𝗨𝗜𝗡𝗧𝗢 𝗚𝗢𝗟 en ocho partidos con ‘los felinos’.  .@SofascoreLA valoró su actuación en 7.4 siendo uno de los mejores de su equipo y consolidándose como uno de los ‘9’  más efectivos del fútbol mexicano.   “¡𝗧𝗥𝗔𝗡𝗤𝗨𝗜𝗟𝗢𝗦, 𝗬𝗢 𝗘𝗦𝗧𝗢𝗬 𝗔𝗤𝗨𝗜́!” — 🇻🇪  #VenEx #LigaMX #Cádiz #SoloVenex   | 📸 @clubleonfc | Sep 22, 2024 · 1:10 AM UTC @SOLOVENEX_ 252 0 26 0 https://tw

In [142]:
table_text = df.to_csv(index=False)

In [108]:
#df2 = pd.read_csv('C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\scrapingcongemini\\venex\\info_solovenex_tweets.csv', header=None)
df2 = df2.astype(str)  # Convert all values to strings
cadena_unica = ' '.join(df2.values.flatten())
print(cadena_unica)

https://twitter.com/SOLOVENEX_/status/1836788008357417004#m ‘𝗦𝗔𝗩𝗔’: ¡𝗘𝗟 𝗠𝗔́𝗦 𝗩𝗔𝗟𝗜𝗢𝗦𝗢! 🇻🇪  @QSavarino10 fue elegido el #HeroOfTheMatch este miércoles en el 0-0 entre @Botafogo y @SaoPauloFC por la ida de los cuartos de final de la Libertadores.   El maracucho fue titular y jugó completo en el duelo ante el equipo de Nahuel Ferraresi:   ➤ 0️⃣3️⃣ pases claves. ➤ 0️⃣1️⃣ tiro a puerta (remate al palo)  ➤ 2️⃣1️⃣ pases precisos de 25 (84%) ➤ 9️⃣0️⃣ minutos disputados.   𝗗𝗜𝗙𝗘𝗥𝗘𝗡𝗖𝗜𝗔𝗟 y pieza 𝗖𝗟𝗔𝗩𝗘 = #JS10 ⚡️  #VenEx #Savarino #Libertadores #SoloVenex  | 📸 @Libertadores | 📊 @idiomafutve | Sep 19, 2024 · 3:22 PM UTC @SOLOVENEX_ 7 0 0 0 https://twitter.com/LAFC/status/1836616613468213733#m 62' | GOAAALLL DAVID MARTÍNEZ  #LAFCvATX 1-1 Sep 19, 2024 · 4:01 AM UTC @LAFC 184 1 15 11 https://twitter.com/SOLOVENEX_/status/1836731886330126485#m 𝗗𝗔𝗩𝗜𝗗 𝗠𝗔𝗥𝗧𝗜́𝗡𝗘𝗭 anotó este miércoles su segundo gol desde que hace vida en la @MLS. Lo hizo en el 1-1 entre su @LAFC y Austin FC.   El ex Monagas ingresó al 60’ y 

In [106]:
table_text2 = df2.to_csv(index=False)

Este seria con el csv de santi que tiene hasta el 17 de sept...me fijo si paso todo a cadena de texto


In [109]:
#contexto3 = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {cadena_unica}"""
contexto3 = f"""sos un agente de social media y necesito que analices este texto que contiene Tweets que te paso a continuación:{cadena_unica}.
Ademas tiene el link del tweet, la fecha en "date", y user, likes, retweets, quotes y comments. 

**Enfoca tu análisis en los jugadores de fútbol mencionados. Para cada jugador, identifica una unica vez:**
* **Nombre completo:**
* **País de origen:** si dice de qque origen es o su nacionalidad.
* **Equipo actual:** si se menciona, sino poner "incompleto"
* **Cualquier referencia a su ascendencia o raíces (e.g., "venezolano de ascendencia italiana", "padre italiano, madre venezolana", "madre inglesa, padre venezolano, nacido en brasil").**
En esta columna buscamos saber si tiene descendencia de otro pais, es decir nos interesa saber la nacionalidad de su madre y/o su padre.
* **Link del tweet
* **Fecha
* **Citar la parte del tweet en donde encontraste la informacion solicitada respecto a la ascendencia y nacionalidad unicamente, ponerla con comillas y si hay varias partes separarlas por coma.

**Considera el siguiente ejemplo de tweet y cómo extraer la información:**
* "Messi marcó un golazo y demostró por qué es el mejor argentino de todos los tiempos en el Miami FC. El hijo de los Rosarinos sigue brillando en la MLS."
INFORMACION= Lionel Messi, Argentino, Miami FC, Padre Rosarino, Madre Rosarina, [Link del tweet], [Fecha], Parte del tweet= "argentino","hijo de los Rosarinos".

**Si no se encuentra información específica sobre la nacionalidad o ascendencia, indica "No se encontró información".**
"""

pregunta3 = f"""Necesito que con la informacion contenida en:{cadena_unica} y teniendo en cuenta las instrucciones en:{contexto3} realices una tabla que tenga las siguientes columnas:
- Nombre completo
- País de origen
- Equipo actual (si se menciona)
- Cualquier referencia a su ascendencia o raíces
- Link del tweet
- Fecha, que corresponde a la parte de la columna date de la tabla, es una sola por fila.
- Citar la o las parte del tweet en donde encontraste la informacion solicitada, ponerla con comillas y si hay varias partes separarlas por coma. 
En total tiene que haber 7 columnas por fila.
Tiene que haber una fila por jugador. No se puede repetir.
No repitas jugadores en la tabla, si un jugador aparece mas de una vez en los tweets mostra el que tenga mas informacion relevante dentro de lo que estamos buscando.
Si no encontras la informacion responde que no esta."""
# Prompt con el contexto y la pregunta
prompt3 = f"{contexto3}\n\n{pregunta3}"

# Generar la respuesta utilizando el prompt actualizado
responses3 = model.generate_content(prompt3, generation_config=generation_config, stream=True)

full_full_text44 = ""
for response3 in responses3:
    full_full_text44 += response3.text

print(full_full_text44)

## Análisis de Tweets sobre Jugadores de Fútbol Venezolanos

| Nombre completo | País de origen | Equipo actual | Cualquier referencia a su ascendencia o raíces | Link del tweet | Fecha | Parte del tweet |
|---|---|---|---|---|---|---|
| Jefferson Savarino | Venezolano | Botafogo | No se encontró información | https://twitter.com/SOLOVENEX_/status/1836788008357417004#m | Sep 19, 2024 | "El maracucho fue titular" |
| David Martínez | Venezolano | LAFC | No se encontró información | https://twitter.com/SOLOVENEX_/status/1836731886330126485#m | Sep 19, 2024 | "El ex Monagas ingresó al 60’" |
| Juan Arango Jr. | Venezolano | Girona | "El hijo del mítico 18" | https://twitter.com/SOLOVENEX_/status/1836458309840199818#m | Sep 18, 2024 |  |
| Yordan Osorio | Venezolano | Parma | "El barinés" | https://twitter.com/SOLOVENEX_/status/1836146859422945773#m | Sep 17, 2024 |  |
| Daniel Maldini | Venezolano | Monza | "Es hijo de Adriana Fossa, ex modelo venezolana y esposa de Paolo Maldini." | http

In [110]:

# Suponiendo que full_full_text44 contiene el texto que deseas procesar
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_full_text44)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df44 = pd.DataFrame(matches)

# Elimina filas que consisten solo en '---'
df44 = df44[~df44.eq('---').all(axis=1)]

# Asigna los nombres de columna usando la primera fila
df44.columns = df44.iloc[0]  # La primera fila se convierte en nombres de columna
df44 = df44[1:]  # Elimina la fila que ahora es el encabezado

# Resetea el índice
df44.reset_index(drop=True, inplace=True)


In [111]:
df44.head()

,Nombre completo,País de origen,Equipo actual,Cualquier referencia a su ascendencia o raíces,Link del tweet,Fecha,Parte del tweet
0,Jefferson Savarino,Venezolano,Botafogo,No se encontró información,https://twitter.com/SOLOVENEX_/status/18367880...,"Sep 19, 2024","""El maracucho fue titular"""
1,David Martínez,Venezolano,LAFC,No se encontró información,https://twitter.com/SOLOVENEX_/status/18367318...,"Sep 19, 2024","""El ex Monagas ingresó al 60’"""
2,Juan Arango Jr.,Venezolano,Girona,"""El hijo del mítico 18""",https://twitter.com/SOLOVENEX_/status/18364583...,"Sep 18, 2024",
3,Yordan Osorio,Venezolano,Parma,"""El barinés""",https://twitter.com/SOLOVENEX_/status/18361468...,"Sep 17, 2024",
4,Daniel Maldini,Venezolano,Monza,"""Es hijo de Adriana Fossa, ex modelo venezolan...",https://twitter.com/SOLOVENEX_/status/18356697...,"Sep 16, 2024",


In [112]:
df44.to_csv('tabla_df2_final_string.csv', index=False)

como csv, modifico el prompt doy mas detalle

In [113]:
#contexto3 = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {cadena_unica}"""
contexto = f"""sos un agente de social media y necesito que analices este texto que contiene Tweets que te paso a continuación:{table_text2}.
Ademas tiene el link del tweet, la fecha en "date", y user, likes, retweets, quotes y comments. 

**Enfoca tu análisis en los jugadores de fútbol mencionados. Para cada jugador, identifica una unica vez:**
* **Nombre completo:**
* **País de origen:** si dice de qque origen es o su nacionalidad.
* **Equipo actual:** si se menciona, sino poner "incompleto"
* **Cualquier referencia a su ascendencia o raíces (e.g., "venezolano de ascendencia italiana", "padre italiano, madre venezolana", "madre inglesa, padre venezolano, nacido en brasil").**
En esta columna buscamos saber si tiene descendencia de otro pais, es decir nos interesa saber la nacionalidad de su madre y/o su padre.
* **Link del tweet
* **Fecha
* **Citar la parte del tweet en donde encontraste la informacion solicitada respecto a la ascendencia y nacionalidad unicamente, ponerla con comillas y si hay varias partes separarlas por coma.

**Considera el siguiente ejemplo de tweet y cómo extraer la información:**
* "Messi marcó un golazo y demostró por qué es el mejor argentino de todos los tiempos en el Miami FC. El hijo de los Rosarinos sigue brillando en la MLS."
INFORMACION= Lionel Messi, Argentino, Miami FC, Padre Rosarino, Madre Rosarina, [Link del tweet], [Fecha], Parte del tweet= "argentino","hijo de los Rosarinos".

**Si no se encuentra información específica sobre la nacionalidad o ascendencia, indica "No se encontró información".**
"""

pregunta = f"""Necesito que con la informacion contenida en:{table_text2} y teniendo en cuenta las instrucciones en:{contexto} realices una tabla que tenga las siguientes columnas:
- Nombre completo
- País de origen
- Equipo actual (si se menciona)
- Cualquier referencia a su ascendencia o raíces
- Link del tweet
- Fecha, que corresponde a la parte de la columna date de la tabla, es una sola por fila.
- Citar la o las parte del tweet en donde encontraste la informacion solicitada, ponerla con comillas y si hay varias partes separarlas por coma. 
En total tiene que haber 7 columnas por fila.
Tiene que haber una fila por jugador. No se puede repetir.
No repitas jugadores en la tabla, si un jugador aparece mas de una vez en los tweets mostra el que tenga mas informacion relevante dentro de lo que estamos buscando.
Si no encontras la informacion responde que no esta."""
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

full_full_text = ""
for response in responses:
    full_full_text += response.text

print(full_full_text)

## Análisis de Tweets sobre Jugadores de Fútbol Venezolanos

| Nombre completo | País de origen | Equipo actual | Cualquier referencia a su ascendencia o raíces | Link del tweet | Fecha | Parte del tweet |
|---|---|---|---|---|---|---|
| Savarino, Jefferson | Venezolano | Botafogo | No se encontró información | https://twitter.com/SOLOVENEX_/status/1836788008357417004#m | Sep 19, 2024 · 3:22 PM UTC | "El maracucho fue titular" |
| Martínez, David | Venezolano | LAFC | No se encontró información | https://twitter.com/LAFC/status/1836616613468213733#m | Sep 19, 2024 · 4:01 AM UTC | No se encontró información |
| Arango Jr., Juan | Venezolano | Girona | "El hijo del mítico 18" | https://twitter.com/SOLOVENEX_/status/1836458309840199818#m | Sep 18, 2024 · 5:32 PM UTC |  |
| Osorio, Yordan | Venezolano | Parma | "El barinés" | https://twitter.com/SOLOVENEX_/status/1836146859422945773#m | Sep 17, 2024 · 8:54 PM UTC |  |
| Maldini, Daniel | Venezolano | Monza | "Es hijo de Adriana Fossa, ex m

In [91]:
# Suponiendo que full_full_text44 contiene el texto que deseas procesar
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_full_text)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df4 = pd.DataFrame(matches)

# Elimina filas que consisten solo en '---'
df4 = df4[~df4.eq('---').all(axis=1)]

# Asigna los nombres de columna usando la primera fila
df4.columns = df4.iloc[0]  # La primera fila se convierte en nombres de columna
df4 = df4[1:]  # Elimina la fila que ahora es el encabezado

# Resetea el índice
df4.reset_index(drop=True, inplace=True)

In [92]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 7 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Nombre completo                                 346 non-null    object
 1   País de origen                                  346 non-null    object
 2   Equipo actual                                   346 non-null    object
 3   Cualquier referencia a su ascendencia o raíces  346 non-null    object
 4   Link del tweet                                  346 non-null    object
 5   Fecha                                           346 non-null    object
 6   Parte del tweet                                 346 non-null    object
dtypes: object(7)
memory usage: 19.1+ KB


In [93]:
df4.to_csv('tabla_df2_final_table.csv', index=False)

Ahora con el que tiene fechas mas actuales en el df1...o df..primero con string


In [124]:
#contexto3 = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {cadena_unica}"""
contexto31 = f"""sos un agente de social media y necesito que analices este texto que contiene Tweets que te paso a continuación:{cadena_unica1}.
Ademas tiene el link del tweet, la fecha en "date", y user, likes, retweets, quotes y comments. 

**Enfoca tu análisis en los jugadores de fútbol mencionados. Para cada jugador, identifica una unica vez:**
* **Nombre completo:**
* **País de origen:** si dice de qque origen es o su nacionalidad.
* **Equipo actual:** si se menciona, sino poner "incompleto"
* **Cualquier referencia a su ascendencia o raíces (e.g., "venezolano de ascendencia italiana", "padre italiano, madre venezolana", "madre inglesa, padre venezolano, nacido en brasil").**
En esta columna buscamos saber si tiene descendencia de otro pais, es decir nos interesa saber la nacionalidad de su madre y/o su padre.
* **Link del tweet
* **Fecha
* **Citar la parte del tweet en donde encontraste la informacion solicitada respecto a la ascendencia y nacionalidad unicamente, ponerla con comillas y si hay varias partes separarlas por coma.

**Considera el siguiente ejemplo de tweet y cómo extraer la información:**
* "Messi marcó un golazo y demostró por qué es el mejor argentino de todos los tiempos en el Miami FC. El hijo de los Rosarinos sigue brillando en la MLS."
INFORMACION= Lionel Messi, Argentino, Miami FC, Padre Rosarino, Madre Rosarina, [Link del tweet], [Fecha], Parte del tweet= "argentino","hijo de los Rosarinos".

**Si no se encuentra información específica sobre la nacionalidad o ascendencia, indica "No se encontró información".**
"""

pregunta31 = f"""Necesito que con la informacion contenida en:{cadena_unica1} y teniendo en cuenta las instrucciones en:{contexto31} realices una tabla que tenga las siguientes columnas:
- Nombre completo
- País de origen
- Equipo actual (si se menciona)
- Cualquier referencia a su ascendencia o raíces
- Link del tweet
- Fecha, que corresponde a la parte de la columna date de la tabla, es una sola por fila.
- Citar la o las parte del tweet en donde encontraste la informacion solicitada, ponerla con comillas y si hay varias partes separarlas por coma. 
En total tiene que haber 7 columnas por fila.
Tiene que haber una fila por jugador. No se puede repetir.
No repitas jugadores en la tabla, si un jugador aparece mas de una vez en los tweets mostra el que tenga mas informacion relevante dentro de lo que estamos buscando.
Si no encontras la informacion responde que no esta."""
# Prompt con el contexto y la pregunta
prompt31 = f"{contexto31}\n\n{pregunta31}"

# Generar la respuesta utilizando el prompt actualizado
responses31 = model.generate_content(prompt31, generation_config=generation_config, stream=True)

full_full_text441 = ""
for response31 in responses31:
    full_full_text441 += response31.text

print(full_full_text441)

## Análisis de Tweets: Jugadores de Fútbol Venezolanos

| Nombre completo | País de origen | Equipo actual | Cualquier referencia a su ascendencia o raíces | Link del tweet | Fecha | Parte del tweet |
|---|---|---|---|---|---|---|
| Jon Mikel Aramburu | Venezolano | Real Sociedad | No se encontró información | https://twitter.com/SOLOVENEX_/status/1837614128396972102#m | Sep 21, 2024 | "venezolano" |
| Jhonder Cádiz | Venezolano | Club León | "delantero caraqueño" | https://twitter.com/SOLOVENEX_/status/1837660758093844989#m | Sep 22, 2024 | No se encontró información |
| Salomón Rondón | Venezolano | Pachuca | "venezolano", "El gladiador venezolano" | https://twitter.com/SOLOVENEX_/status/1837317464524476486#m | Sep 21, 2024 | No se encontró información |
| Darwin Machís | Venezolano | Real Valladolid | No se encontró información | https://twitter.com/SOLOVENEX_/status/1837614128396972102#m | Sep 21, 2024 | No se encontró información |
| Juan Arango Jr. | Venezolano | Girona | "El hij

In [125]:
# Suponiendo que full_full_text44 contiene el texto que deseas procesar
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_full_text441)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df441 = pd.DataFrame(matches)

# Elimina filas que consisten solo en '---'
df441 = df441[~df441.eq('---').all(axis=1)]

# Asigna los nombres de columna usando la primera fila
df441.columns = df441.iloc[0]  # La primera fila se convierte en nombres de columna
df441 = df441[1:]  # Elimina la fila que ahora es el encabezado

# Resetea el índice
df441.reset_index(drop=True, inplace=True)

In [126]:
df441.head()

,Nombre completo,País de origen,Equipo actual,Cualquier referencia a su ascendencia o raíces,Link del tweet,Fecha,Parte del tweet
0,Jon Mikel Aramburu,Venezolano,Real Sociedad,No se encontró información,https://twitter.com/SOLOVENEX_/status/18376141...,"Sep 21, 2024","""venezolano"""
1,Jhonder Cádiz,Venezolano,Club León,"""delantero caraqueño""",https://twitter.com/SOLOVENEX_/status/18376607...,"Sep 22, 2024",No se encontró información
2,Salomón Rondón,Venezolano,Pachuca,"""venezolano"", ""El gladiador venezolano""",https://twitter.com/SOLOVENEX_/status/18373174...,"Sep 21, 2024",No se encontró información
3,Darwin Machís,Venezolano,Real Valladolid,No se encontró información,https://twitter.com/SOLOVENEX_/status/18376141...,"Sep 21, 2024",No se encontró información
4,Juan Arango Jr.,Venezolano,Girona,"""El hijo del mítico 18""",https://twitter.com/SOLOVENEX_/status/18364583...,"Sep 18, 2024",No se encontró información


In [131]:
df441.tail()

,Nombre completo,País de origen,Equipo actual,Cualquier referencia a su ascendencia o raíces,Link del tweet,Fecha,Parte del tweet
109,Renné Rivas,Venezolano,Al-Taawoun,No se encontró información,https://twitter.com/SOLOVENEX_/status/18289359...,"Aug 28, 2024",No se encontró información
110,Bryant Ortega,Venezolano,Al-Ittihad,No se encontró información,https://twitter.com/SOLOVENEX_/status/18289359...,"Aug 28, 2024",No se encontró información
111,Bianneider Tamayo,Venezolano,U de Chile,No se encontró información,https://twitter.com/SOLOVENEX_/status/18289359...,"Aug 28, 2024",No se encontró información
112,Kevin Kelsy,Venezolano,FC Cincinnati,"""delantero valenciano""",https://twitter.com/SOLOVENEX_/status/18300665...,"Sep 1, 2024",No se encontró información
113,Luis Guerra,Venezolano,incompleto,No se encontró información,https://twitter.com/SOLOVENEX_/status/18299429...,"Aug 31, 2024",No se encontró información


In [129]:
df441.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 7 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Nombre completo                                 114 non-null    object
 1   País de origen                                  114 non-null    object
 2   Equipo actual                                   114 non-null    object
 3   Cualquier referencia a su ascendencia o raíces  114 non-null    object
 4   Link del tweet                                  114 non-null    object
 5   Fecha                                           114 non-null    object
 6   Parte del tweet                                 114 non-null    object
dtypes: object(7)
memory usage: 6.4+ KB


In [130]:
df441.to_csv('tabla_df_final_string.csv', index=False)


con csv el que tiene la fecha hasta hoy

In [143]:
contexto5 = f"""sos un agente de social media y necesito que analices este texto que contiene Tweets que te paso a continuación:{table_text}.
Ademas tiene el link del tweet, la fecha en "date", y user, likes, retweets, quotes y comments. 

**Enfoca tu análisis en los jugadores de fútbol mencionados. Para cada jugador, identifica una unica vez:**
* **Nombre completo:**
* **País de origen:** si dice de qque origen es o su nacionalidad.
* **Equipo actual:** si se menciona, sino poner "incompleto"
* **Cualquier referencia a su ascendencia o raíces (e.g., "venezolano de ascendencia italiana", "padre italiano, madre venezolana", "madre inglesa, padre venezolano, nacido en brasil").**
En esta columna buscamos saber si tiene descendencia de otro pais, es decir nos interesa saber la nacionalidad de su madre y/o su padre.
* **Link del tweet
* **Fecha
* **Citar la parte del tweet en donde encontraste la informacion solicitada respecto a la ascendencia y nacionalidad unicamente, ponerla con comillas y si hay varias partes separarlas por coma.

**Considera el siguiente ejemplo de tweet y cómo extraer la información:**
* "Messi marcó un golazo y demostró por qué es el mejor argentino de todos los tiempos en el Miami FC. El hijo de los Rosarinos sigue brillando en la MLS."
INFORMACION= Lionel Messi, Argentino, Miami FC, Padre Rosarino, Madre Rosarina, [Link del tweet], [Fecha], Parte del tweet= "argentino","hijo de los Rosarinos".

**Si no se encuentra información específica sobre la nacionalidad o ascendencia, indica "No se encontró información".**
"""

pregunta5 = f"""Necesito que con la informacion contenida en:{table_text} y teniendo en cuenta las instrucciones en:{contexto5} realices una tabla que tenga las siguientes columnas:
- Nombre completo
- País de origen
- Equipo actual (si se menciona)
- Cualquier referencia a su ascendencia o raíces
- Link del tweet
- Fecha, que corresponde a la parte de la columna date de la tabla, es una sola por fila.
- Citar la o las parte del tweet en donde encontraste la informacion solicitada, ponerla con comillas y si hay varias partes separarlas por coma. 
En total tiene que haber 7 columnas por fila.
Tiene que haber una fila por jugador. No se puede repetir.
No repitas jugadores en la tabla, si un jugador aparece mas de una vez en los tweets mostra el que tenga mas informacion relevante dentro de lo que estamos buscando.
Si no encontras la informacion responde que no esta."""
# Prompt con el contexto y la pregunta
prompt5 = f"{contexto5}\n\n{pregunta5}"

# Generar la respuesta utilizando el prompt actualizado
responses5 = model.generate_content(prompt5, generation_config=generation_config, stream=True)

full_full_text5 = ""
for response5 in responses5:
    full_full_text5 += response5.text

print(full_full_text5)

##  Tabla de Jugadores Venezolanos

| Nombre completo | País de origen | Equipo actual | Cualquier referencia a su ascendencia o raíces | Link del tweet | Fecha | Citar la parte del tweet en donde encontraste la informacion solicitada |
|---|---|---|---|---|---|---|
|  Rómulo Otero | Venezolano | Santos FC | No se encontró información | https://twitter.com/SOLOVENEX_/status/1750956186168402081 | Jan 26, 2024 | "Soteldo es el nuevo #7 del Gremio! 🇻🇪   #VenEx #Soteldo #Gremio #SoloVenex   | 📸 Imagen cortesía |" |
|  Yeferson Soteldo | Venezolano | Gremio | No se encontró información | https://twitter.com/SOLOVENEX_/status/1750956186168402081 | Jan 26, 2024 | "Soteldo es el nuevo #7 del Gremio! 🇻🇪   #VenEx #Soteldo #Gremio #SoloVenex   | 📸 Imagen cortesía |" |
|  Yordan Osorio | Venezolano | Parma | No se encontró información | https://twitter.com/SOLOVENEX_/status/1750912485861793868 | Jan 26, 2024 | "𝗢𝗙𝗜𝗖𝗜𝗔𝗟: Yordan Osorio hasta 𝟮𝟬𝟮𝟱 🇮🇹  ¡Pulgar arriba para el central barinés! 👍🏻  • Su 

In [144]:
# Suponiendo que full_full_text44 contiene el texto que deseas procesar
matches5 = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_full_text5)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df5 = pd.DataFrame(matches5)

# Elimina filas que consisten solo en '---'
df5 = df5[~df5.eq('---').all(axis=1)]

# Asigna los nombres de columna usando la primera fila
df5.columns = df5.iloc[0]  # La primera fila se convierte en nombres de columna
df5 = df5[1:]  # Elimina la fila que ahora es el encabezado

# Resetea el índice
df5.reset_index(drop=True, inplace=True)

In [145]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column                                                                   Non-Null Count  Dtype 
---  ------                                                                   --------------  ----- 
 0   Nombre completo                                                          70 non-null     object
 1   País de origen                                                           70 non-null     object
 2   Equipo actual                                                            70 non-null     object
 3   Cualquier referencia a su ascendencia o raíces                           70 non-null     object
 4   Link del tweet                                                           70 non-null     object
 5   Fecha                                                                    70 non-null     object
 6   Citar la parte del tweet en donde encontraste la informacion solicitada  70 non-null 

In [146]:
df5.to_csv('tabla_df_final_table.csv', index=False)